<style>
    body, * {
        direction: rtl !important;
        text-align: right !important;
    }
</style>
מקור: [Spark By Examples - Convert PySpark RDD to DataFrame
](https://sparkbyexamples.com/pyspark/convert-pyspark-rdd-to-dataframe/)

# 🔄 המרת PySpark RDD ל-DataFrame

**תאריך:** 27 במרץ, 2024  
**זמן קריאה משוער:** 6 דקות

---

## 🎯 מבוא

ב-PySpark, פונקציית `toDF()` של ה-RDD משמשת להמרת RDD ל-DataFrame. 

**למה להמיר RDD ל-DataFrame?**

אנו צריכים להמיר RDD ל-DataFrame מכיוון ש-DataFrame מספק יותר יתרונות מאשר RDD. לדוגמה:

- 📊 **DataFrame** הוא אוסף מבוזר של נתונים מאורגן לעמודות עם שמות
- 🗄️ דומה לטבלאות במסדי נתונים
- ⚡ מספק אופטימיזציה ושיפורי ביצועים
- 🛠️ API נוח יותר לעבודה עם נתונים מובנים

### 📑 תוכן העניינים:

1. יצירת PySpark RDD
2. המרת PySpark RDD ל-DataFrame
   - שימוש ב-`toDF()`
   - שימוש ב-`createDataFrame()`
   - שימוש ב-`createDataFrame()` עם StructType schema
3. דוגמה מלאה

## 1️⃣ יצירת PySpark RDD

ראשית, בואו ניצור RDD על ידי העברת אובייקט רשימת Python לפונקציה `sparkContext.parallelize()`.

נצטרך את אובייקט ה-`rdd` הזה לכל הדוגמאות שלנו להלן.

### 🔧 הגדרת הסביבה ויצירת RDD

In [ ]:
from pyspark.sql import SparkSession

# יצירת SparkSession
spark = SparkSession.builder.appName("SparkByExamples.com").getOrCreate()

# יצירת נתונים לדוגמה
dept = [("Finance", 10), ("Marketing", 20), ("Sales", 30), ("IT", 40)]

# יצירת RDD
rdd = spark.sparkContext.parallelize(dept)

**הסבר:**

ב-PySpark, כאשר יש לך נתונים ברשימה, כלומר יש לך אוסף של נתונים בזיכרון ה-driver של PySpark, כאשר אתה יוצר RDD, האוסף הזה הולך להיות **מקובל** (parallelized).

המשמעות היא שהנתונים מתפזרים על פני מספר workers כדי לאפשר עיבוד מקבילי.

## 2️⃣ המרת PySpark RDD ל-DataFrame

המרת PySpark RDD ל-DataFrame יכולה להתבצע באמצעות `toDF()`, `createDataFrame()`. 

בחלק זה, אסביר את שתי השיטות האלה.

### 2.1 שימוש בפונקציה `rdd.toDF()`

PySpark מספק פונקציה `toDF()` ב-RDD שניתן להשתמש בה כדי להמיר RDD ל-DataFrame.

In [ ]:
# המרה פשוטה ל-DataFrame
df = rdd.toDF()
df.printSchema()
df.show(truncate=False)

**תוצאה צפויה:**

```
root
 |-- _1: string (nullable = true)
 |-- _2: long (nullable = true)

+----------+---+
|_1        |_2 |
+----------+---+
|Finance   |10 |
|Marketing |20 |
|Sales     |30 |
|IT        |40 |
+----------+---+
```

כברירת מחדל, פונקציה `toDF()` יוצרת שמות עמודות כ-"_1" ו-"_2". קטע הקוד הזה מניב את ה-schema להלן.

### 🏷️ הוספת שמות עמודות מותאמים

ל-`toDF()` יש signature נוסף שמקבל ארגומנטים כדי להגדיר שמות עמודות כפי שמוצג להלן.

In [ ]:
# המרה ל-DataFrame עם שמות עמודות מותאמים
deptColumns = ["dept_name", "dept_id"]
df2 = rdd.toDF(deptColumns)
df2.printSchema()
df2.show(truncate=False)

**תוצאה צפויה:**

```
root
 |-- dept_name: string (nullable = true)
 |-- dept_id: long (nullable = true)

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|Finance  |10     |
|Marketing|20     |
|Sales    |30     |
|IT       |40     |
+---------+-------+
```

זה מדפיס את ה-schema למטה.

## 2.2 שימוש בפונקציה `createDataFrame()` של PySpark

מחלקה `SparkSession` מספקת שיטה `createDataFrame()` ליצירת DataFrame והיא מקבלת אובייקט `rdd` כארגומנט.

In [ ]:
# שימוש ב-createDataFrame
deptDF = spark.createDataFrame(rdd, schema=deptColumns)
deptDF.printSchema()
deptDF.show(truncate=False)

**הסבר:**

שיטה זו מניבה את אותה תוצאה כמו למעלה - DataFrame עם שמות העמודות שהגדרנו.

## 2.3 שימוש ב-`createDataFrame()` עם StructType schema

כאשר אתה מסיק את ה-schema, כברירת מחדל טיפוס הנתונים של העמודות נגזר מהנתונים ומגדיר nullable ל-true לכל העמודות.

אנחנו יכולים לשנות את ההתנהגות הזו על ידי אספקת **schema** באמצעות **StructType** – שבו אנחנו יכולים לציין שם עמודה, טיפוס נתונים ו-nullable לכל field/column.

### 📐 הגדרת Schema מפורש עם StructType

אם אתה רוצה לדעת עוד על StructType, אנא עבור דרך **how to use StructType and StructField to define the custom schema** (מדריך נפרד).

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType

# הגדרת schema מפורש
deptSchema = StructType([
    StructField("dept_name", StringType(), True),
    StructField("dept_id", StringType(), True)
])

# יצירת DataFrame עם Schema מפורש
deptDF1 = spark.createDataFrame(rdd, schema=deptSchema)
deptDF1.printSchema()
deptDF1.show(truncate=False)

**יתרונות שימוש ב-StructType:**

- 🎯 **שליטה מלאה** על טיפוסי הנתונים
- ✅ **הגדרת nullable** לכל עמודה
- 📋 **תיעוד ברור** של מבנה הנתונים
- 🔒 **אכיפת סכימה** מוקדמת

זה גם מניב את אותה תוצאה.

## 📌 סיכום

במאמר זה, למדת כיצד להמיר PySpark RDD ל-DataFrame. 

נצטרך זאת לעתים קרובות בזמן עבודה ב-PySpark מכיוון שאלה מספקים אופטימיזציה וביצועים מעל RDD.

### ✅ סיכום השיטות:

| שיטה | תיאור | מתי להשתמש |
|------|-------|------------|
| **toDF()** | המרה פשוטה ללא שמות | לבדיקות מהירות |
| **toDF(columns)** | המרה עם שמות עמודות | כשיש שמות פשוטים |
| **createDataFrame()** | שיטה מפורשת | לקוד ברור יותר |
| **createDataFrame() + StructType** | שליטה מלאה על Schema | לייצור, כשצריך טיפוסי נתונים מדויקים |

### 🎯 המלצות:

- ✅ **לייצור**: השתמש ב-StructType schema להגדרה מפורשת
- ✅ **לפיתוח**: toDF() מהיר ונוח
- ✅ **לביצועים**: DataFrame תמיד עדיף על RDD

**Happy Learning !!** 🎓

## 📚 מאמרים קשורים

- **PySpark – Create DataFrame with Examples**
- **PySpark Convert DataFrame to RDD**
- **PySpark Create RDD with Examples**
- **PySpark RDD Actions with examples**
- **PySpark Row using on DataFrame and RDD**
- **PySpark – Create an Empty DataFrame & RDD**
- **PySpark parallelize() – Create RDD from a list data**
- **PySpark Replace Column Values in DataFrame**
- **PySpark RDD Actions with examples**

## 🏷️ תגיות (Tags)

`DATAFRAME` `DATASET` `RDD` `SPARK.CREATEDATAFRAME` `TODF()`

## 🔗 מקורות ומידע נוסף

**מקור המדריך:**
- SparkByExamples.com

**תיעוד רשמי:**
- [PySpark RDD Documentation](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.html)
- [PySpark DataFrame Documentation](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/dataframe.html)
- [Apache Spark SQL Guide](https://spark.apache.org/docs/latest/sql-programming-guide.html)

**קוד מלא:**
- הקוד המלא ניתן להורדה מ-GitHub (קישור במקור)

---

**💻 בהצלחה בלימוד PySpark!**

**✨ Happy Learning !!**